<figure>
  <img src="https://github.com/v-iashin/video_features/raw/master/docs/_assets/r21d.png" width="300" />
</figure>

The `video_features` library allows you to extract features from
raw videos in parallel with multiple GPUs.
It supports several extractors that capture visual appearance,
optical flow, and audio features. See more details in the
[GitHub repository](https://github.com/v-iashin/video_features).

See more feature extraction examples in colaboratory notebooks:
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Zd7r8uKGLGSxlil4PPnXk_4I3KOsjPpO?usp=sharing) – CLIP
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1HUlYcOJf_dArOcAaR9jaQHuM5CAZiNZc?usp=sharing) – S3D
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1LKoytZmNxtC-EuCp7pHDM6sFvK1XdwlW?usp=sharing) – I3D
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1csJgkVQ3E2qOyVlcOM-ACHGgPBBKwE2Y?usp=sharing) – R(2+1)D
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/18I95Rn1B3a2ISfD9b-o4o93m3XuHbcIY?usp=sharing) – RAFT
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/17VLdf4abQT2eoMjc6ziJ9UaRaOklTlP0?usp=sharing) – ResNet
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1r_8OnmwXKwmH0n4RxBfuICVBgpbJt_Fs?usp=sharing) – VGGish
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/16QEwNMqiwqlmBhJCJmNeeEP8gitom0I-?usp=sharing) – [timm](https://huggingface.co/timm) models

In [1]:
import os
! git clone https://github.com/v-iashin/video_features.git
! pip install omegaconf==2.0.6 av==10.0

In [3]:
%cd video_features
%pwd

In [4]:
import os
print(os.getcwd())
os.chdir('F:\Machine_learning\P_3\\video_features')
print(os.getcwd())

In [5]:
from video_features.models.r21d.extract_r21d import ExtractR21D
from video_features.utils.utils import build_cfg_path
from omegaconf import OmegaConf
import pandas as pd
import torch

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
torch.cuda.get_device_name(0)

# Add all the videos to process the features

In [2]:
import  os
import numpy as np

import shutil
print(os.getcwd())
os.chdir('F:\\Machine_learning\\P_3') #use your local machine path 


source_dir = 'train_subset'
target_dir1 = 'train_subset_1' #Fabricio
target_dir2 = 'train_subset_2'  #Furro Egus



path = os.getcwd()
files = os.listdir(path+'\\'+source_dir)

print(os.getcwd())





# Create target directories if they don't exist
os.makedirs(target_dir1, exist_ok=True)
os.makedirs(target_dir2, exist_ok=True)

print("Total  train list ",len(files))

# Split the files into two equal parts
midpoint = len(files) // 2
files_part1 = files[:midpoint]
files_part2 = files[midpoint:]

# Move the files to the respective target directories
for file in files_part1:
    shutil.move(os.path.join(source_dir, file), os.path.join(target_dir1, file))

for file in files_part2:
    shutil.move(os.path.join(source_dir, file), os.path.join(target_dir2, file))

print(f'Files have been divided and moved to {target_dir1} and {target_dir2}')



feature_path = "F:\\Machine_learning\\P_3\\train_subset_features" #use your local machine path 
train_path = "F:/Machine_learning/P_3/" + target_dir1 #use your local machine path  also  #Change here into part2 for Egus :3

if len(files_part1) ==0 : #Change here into part2 for Furro Egus :3 
    files_part1 = os.listdir(train_path)


if not os.path.exists(feature_path):
    os.makedirs(feature_path)

train_list = [os.path.join(train_path, f) for f in files_part1]
print("Your new work load is" ,train_path +'----->'+ str(len(train_list)))
if os.access(feature_path, os.W_OK):
    print(f"Tienes permisos de escritura en el directorio: {feature_path}")
else:
    print(f"No tienes permisos de escritura en el directorio: {feature_path}")    
    

In [30]:
#restore the old path 
os.chdir('F:\\Machine_learning\\P_3\\video_features') #use your local machine path 
print(os.getcwd())
print(feature_path) #use your local machine path 

In [31]:
# Cargar y ajustar la configuración
feature_type = 'r21d'
args = OmegaConf.load(build_cfg_path(feature_type))
args.video_paths = train_list
args.model_name = 'r2plus1d_18_16_kinetics'
args.device = device

# Cargar el modelo
extractor = ExtractR21D(args)

# Diccionario para almacenar las características
all_features = {}

# Contador de videos procesados
processed_count = 0

# Extraer características
for video_path in args.video_paths:
    processed_count += 1
    video_name = os.path.basename(video_path)
    
    # Imprimir el video que se está procesando
    #print(f'Extrayendo características para {video_name}')
    
    # Extraer características
    try:
        feature_dict = extractor.extract(video_path)
        
        # Verificar si se generan características
        if not feature_dict:
            print(f"No se generaron características para {video_name}")
        else:
            for k, v in feature_dict.items():
                #print(f'This is key: {k}')
                #print(f'This is value shape: {v.shape}')

                # Guardar las características en el diccionario
                video_key = os.path.basename(video_name).replace('.mp4', '')
                if video_key not in all_features:
                    all_features[video_key] = {}
                    all_features[video_key][k] = v

        # Mostrar el número de videos procesados
        print(f'Videos procesados: {processed_count}')
    
    except RuntimeError as e:
        print(f"Error procesando {video_name}: {e}")

# Guardar todas las características a la memoria secundaria al final


In [32]:
for video_key, features in all_features.items():
    for k, v in features.items():
        output_file = os.path.join(feature_path, f'{video_key}.npy')
        with open(output_file, 'wb') as f:
            np.save(f, v)
        print(f'Guardado: {output_file}')

# Process test_features from videos

In [6]:
test_path = "F:/Machine_learning/P_3/test_subset"  #use your local machine path  also  
test_feature_path = "F:/Machine_learning/P_3/test_subset_features"
test_files = os.listdir(test_path)
test_list = [os.path.join(test_path, f) for f in test_files]
feature_type = 'r21d'
args = OmegaConf.load(build_cfg_path(feature_type))
args.video_paths = test_list
args.model_name = 'r2plus1d_18_16_kinetics'
args.device = device

# Cargar el modelo
extractor = ExtractR21D(args)

# Diccionario para almacenar las características
all_features = {}

# Contador de videos procesados
processed_count = 0

# Extraer características
for video_path in args.video_paths:
    processed_count += 1
    video_name = os.path.basename(video_path)
    
    # Imprimir el video que se está procesando
    #print(f'Extrayendo características para {video_name}')
    
    # Extraer características
    try:
        feature_dict = extractor.extract(video_path)
        
        # Verificar si se generan características
        if not feature_dict:
            print(f"No se generaron características para {video_name}")
        else:
            for k, v in feature_dict.items():
                #print(f'This is key: {k}')
                #print(f'This is value shape: {v.shape}')

                # Guardar las características en el diccionario
                video_key = os.path.basename(video_name).replace('.mp4', '')
                if video_key not in all_features:
                    all_features[video_key] = {}
                    all_features[video_key][k] = v

        # Mostrar el número de videos procesados
        print(f'Videos procesados: {processed_count}')
    
    except RuntimeError as e:
        print(f"Error procesando {video_name}: {e}")


# Save data for testing


In [10]:
import numpy as np
for video_key, features in all_features.items():
    for k, v in features.items():
        output_file = os.path.join(test_feature_path, f'{video_key}.npy')
        with open(output_file, 'wb') as f:
            np.save(f, v)
        print(f'Guardado: {output_file}')

# Cargar data guardada de los video features



In [1]:

# Directorio donde están almacenados los archivos .npy
feature_dir = 'F:\\Machine_learning\\P_3\\train_subset_features'
import os
import numpy as np
# Listar todos los archivos .npy en el directorio
feature_files = [f for f in os.listdir(feature_dir) if f.endswith('.npy')]

# Cargar y procesar cada archivo .npy
all_data = []
all_names = []
for file_name in feature_files:
    file_path = os.path.join(feature_dir, file_name)
    data = np.load(file_path)
    if data.shape == (0,):
        print(f'Skipping {file_name} because it is empty.')
    elif data.shape[1] == 512:
        print(f'Loaded {file_name} with shape: {data.shape}')
        all_data.append(data)
        all_names.append(file_name[0:11])
    else:
        print(f'Skipping {file_name} due to incorrect shape: {data.shape}')

print(f'Loaded {len(all_data)} files.')


In [25]:
%pwd
%cd ..
%pwd

In [12]:
from Data_processing_support.feature_summary_techniques import apply_pca_fixed_components , apply_pca
new_matrix = []
for matrix in all_data:
    m , n_comp =apply_pca(matrix , variance_threshold=0.95 )
    print(n_comp)
    new_matrix.append(m)
print(all_data[0].shape)

 

# PROCESANDO DATA PARA CLUSTERS


In [26]:
# SACANDO MEDIA PARA CADA DATO DE all_data
from Data_processing_support.feature_summary_techniques import summary_by_mean
print("size of data : " , all_data[10223].shape)
print("size of labels : " , len(all_names))
np_mean=summary_by_mean(all_data)
print("New data frame values and categories :" , np_mean.shape)
print(all_names[0:10])
#print("Nuevo data frame sizes " , df_mean.shape)
df = pd.DataFrame(np_mean)
df.to_csv('train_mean.csv')

# Usando Kmeans para procesamiento

In [6]:
# Aplicar KMeans
import pandas as pd
from sklearn.cluster import KMeans , DBSCAN
from Data_processing_support.feature_summary_techniques import add_true_categories_to_data_frame  , get_all_metrics
kmeans  = KMeans(n_clusters=20, random_state=42)
labels = kmeans.fit_predict(np_mean)
df_mean = pd.DataFrame(np_mean)

df_mean['video'] = all_names
df_mean['cluster'] = labels

df_mean = add_true_categories_to_data_frame(df_mean)

print(df_mean[['video', 'category' ,'cluster' ]].head())

true_labels = df_mean['category']
predicted_labels = df_mean['cluster']

get_all_metrics(predicted_labels, true_labels , np_mean)


# Agglomerating method

In [4]:
%pwd

In [5]:
from scipy.cluster.hierarchy import linkage  , dendrogram 
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt

clustering  = AgglomerativeClustering(n_clusters=20)
labels = clustering.fit_predict(np_mean)
df_mean = pd.DataFrame(np_mean)

df_mean['video'] = all_names
df_mean['cluster'] = labels

df_mean = add_true_categories_to_data_frame(df_mean)

print(df_mean[['video', 'category' ,'cluster' ]].head())

true_labels = df_mean['category']
predicted_labels = df_mean['cluster']

get_all_metrics(labels, true_labels , np_mean)



# Sample data
np.random.seed(0)
X = np.random.rand(10, 2)  # 10 points in 2 dimensions

# Perform hierarchical clustering using different linkage methods
Z_single = linkage(X, 'single')


print(Z_single)
plt.figure(figsize=(10, 7))
dendrogram(Z_single)
plt.title('Dendrograma de Agrupamiento Jerárquico')
plt.xlabel('Índice de muestra')
plt.ylabel('Distancia')
plt.show()
    

In [1]:
from Cluster_Models.Agglomerate_method import AgglomerativeMethod  , cluster
import heapq
list = [(3 ,'A'  ,'C'),(5 , 'B' , 'D'),(1,'F' , 'MN'),(6,'E' , 'HI'),(10,'D' , 'L')]

heapq.heapify(list)
heapq.heappush(list, (-1,'MOMOKA :3' , 'coso'))
print(list)

dict = {}

for i, element in zip(range(len(list)), list):
    key = element[1]+element[2]
    print(key)
    dict[key] = i






list[dict['FMN']] =list[0]  
print(list)
heapq.heappop(list)
print(list)
heapq.heappush(list, (-10,'MOMOKA >w<' , 'hiniatura'))
print(list)


    

In [1]:
%pwd


# Pretraining init distance matrix and heap for distances


In [4]:
import pandas as pd
import numpy as np

from Cluster_Models.Agglomerate_method import AgglomerativeMethod 
from scipy.spatial import distance_matrix  ,KDTree 
from scipy.spatial.distance import pdist, squareform

np_mean = pd.read_csv('train_mean.csv')

#data = np.array([[1, 2], [2, 3], [3, 4], [8, 9], [9, 10], [10, 11]])



np_mean = np_mean.to_numpy()
cluster_method = AgglomerativeMethod(data=np_mean  , cluster_number=20)
cluster_method.init_clusters()
cluster_method.init_distance_matrix_heap()





Actual training



In [5]:
cluster_method.fit_a()